In [1]:
import pandas as pd
import numpy as np
import pickle
import tensorflow as tf
from tensorflow.keras.models import load_model

In [2]:
## Load ANN Train Models, OHE, Scaler pickle file
model = load_model('model.h5')

## load the encode and scaler
with open('onehot_encoder_geo.pkl','rb') as file:
    label_encoder_geo = pickle.load(file)

with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [5]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [19]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [20]:
## Transform Gender
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [21]:
geo_encoded = label_encoder_geo.transform(input_df[['Geography']])
geo_encoded_df = pd.DataFrame(geo_encoded.toarray(), columns = label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [23]:
input_data = pd.concat([input_df.drop(['Geography'],axis=1),geo_encoded_df], axis=1)

In [24]:
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [25]:
input_scaled = scaler.transform(input_data)
input_scaled

array([[-0.47154541,  0.90911166,  0.09477172, -0.69844549, -0.29010416,
         0.80510537,  0.63367318,  0.95214374, -0.84805047,  0.98019606,
        -0.57581067, -0.56349184]])

In [26]:
## Predict Churn
prediction = model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 31ms/step


array([[0.03368783]], dtype=float32)

In [28]:
prediction_proba= prediction[0][0]

In [29]:
prediction_proba

0.033687834

In [31]:
if prediction_proba > 0.5 :
    print("The Customer is likely to churn")
else:
    print("The Customer is not likely to churn")

The Customer is not likely to churn
